In [1]:
!pip install mamba-ssm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 27.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 23.9 MB/s eta 0:00:00
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.2-cp310-cp310-linux_x86_64.whl size=343801721 sha256=9bfeb28e28d3c83adf5714542482daaf7a835f3372e4204dbb05e5b3c73f025c
  Stored in directory: /root/.cache/pip/wheels/57/7c/90/9f963468ecc3791e36e388f9e7b4a4e1e3f90fbb340055aa4d
Successfully built mamba-ssm


In [2]:
!pip install pydub
!pip install librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 38.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 39.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 40.0 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install thop

In [1]:
import torch
torch.cuda.is_available()

True

In [ ]:
# Get the total memory of the GPU
total_memory = torch.cuda.get_device_properties(0).total_memory

# Calculate the fraction that corresponds to 1GB
fraction = 1 * 1024**3 / total_memory

# Set the memory fraction for the current process
torch.cuda.set_per_process_memory_fraction(fraction, device=0)

In [5]:
# import tensorflow_datasets as tfds
# # Load data
# version = 3 # just 3 is available
# ds, info = tfds.load(f'speech_commands:0.0.{version}', with_info=True, split='train[:5%]', as_supervised=True)


In [6]:
# import mini dataset for better data loading time
import os
import pathlib
import tensorflow as tf
import numpy as np

# maintain seed for repructablity
np.seed = 42
tf.random.set_seed(42)
torch.manual_seed(0)


DATASET_PATH = 'data/mini_speech_commands'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

2024-08-08 22:16:47.443019: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-08 22:16:47.451778: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-08 22:16:47.465888: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 22:16:47.488654: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 22:16:47.495050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attemptin

182082353/182082353 ━━━━━━━━━━━━━━━━━━━━ 42s 0us/step


In [10]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[(commands != 'README.md') & (commands != '.DS_Store')]
print('Commands:', commands)

Commands: ['no' 'go' 'left' 'down' 'yes' 'right' 'up' 'stop']


In [11]:
num_labels = len(commands)
print('Number of labels:', num_labels)

Number of labels: 8


In [12]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=None,
    validation_split=0.2,
    seed=0,
    output_sequence_length=16000,
    subset='both')

label_names = np.array(train_ds.class_names)
print()
print("label names:", label_names)

Found 8000 files belonging to 8 classes.
Using 6400 files for training.
Using 1600 files for validation.

label names: ['down' 'go' 'left' 'no' 'right' 'stop' 'up' 'yes']


2024-08-08 22:20:55.174457: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
print('Batch Shape:', train_ds.element_spec[0].shape)
print('Label Shape:', train_ds.element_spec[1].shape)

Batch Shape: (16000, None)
Label Shape: ()


In [14]:
print(train_ds.element_spec)

(TensorSpec(shape=(16000, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))


In [15]:
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
from librosa.feature import mfcc

# Define the dataset adapter for smaller dataset:
class TFDatasetAdapter(Dataset):
    def __init__(self, tf_dataset, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations):
        self.tf_dataset = tf_dataset
        self.data = list(tf_dataset)
        self.fixed_length = fixed_length
        self.n_mfcc = n_mfcc
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.augmentations = augmentations

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio, label = self.data[idx]
        audio = audio.numpy()

        # Ensure the audio tensor has the correct shape (1D array)
        if audio.ndim > 1:
            audio = np.squeeze(audio)

        # Apply augmentations if any
        if self.augmentations:
            for aug in self.augmentations:
                audio = aug(audio)

        # Pad or trim the audio to the fixed length
        if len(audio) < self.fixed_length:
            audio = np.pad(audio, (0, self.fixed_length - len(audio)), mode='constant')
        else:
            audio = audio[:self.fixed_length]



        # Create MFCCs from an audio tensor using Librosa.
        audio = audio.astype(np.float32)
        MFCC = mfcc(y=audio, sr=16000, n_mfcc=self.n_mfcc, n_fft=self.n_fft, hop_length=self.hop_length, n_mels=self.n_mels)

        # Remove extra dimension if it exists
        if MFCC.ndim == 3:
            MFCC = MFCC.squeeze(-1)

        return torch.tensor(MFCC, dtype=torch.float32), torch.tensor(label.numpy(), dtype=torch.long)


In [16]:
FREQUENCY = 16000
DURATION = 16000
def add_time_shift_noise_and_align(audio, max_shift_in_ms=100):
    # randomly shift the audio by at most max_shift_in_ms
    max_shift = (max_shift_in_ms * FREQUENCY) // 1000
    time_shift = np.random.randint(0, max_shift)
    future = np.random.randint(0, 2)

    if future == 0:
        audio = np.pad(audio[time_shift:], (0, time_shift), 'constant')
    else:
        audio = np.pad(audio[:-time_shift], (time_shift, 0), 'constant')

    # Ensure the audio tensor has the correct length
    if len(audio) < DURATION:
        audio = np.pad(audio, (DURATION - len(audio), 0), 'constant')

    return audio[:DURATION]

def add_noise(audio, noise_level=0.005):
    noise = np.random.randn(len(audio)) * noise_level
    return audio + noise

In [17]:
augmentations = [
    lambda x: add_time_shift_noise_and_align(x),
    lambda x:add_noise(x,noise_level = 0.01)
]

In [18]:
# Convert the TFDS dataset to a PyTorch Dataset
fixed_length = 16000
n_mfcc = 13
n_fft = 400
hop_length = 160
n_mels = 40
pytorch_train_dataset = TFDatasetAdapter(train_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations)
pytorch_val_dataset = TFDatasetAdapter(val_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations)

2024-08-08 22:21:08.857895: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-08 22:21:09.222942: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [35]:
# Create a DataLoader to feed the data into the model
batch_size = 128
train_loader = DataLoader(pytorch_train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(pytorch_val_dataset, batch_size=batch_size, shuffle=False)

In [36]:
for audio, label in train_loader:
    print(audio.shape, label.shape)
    break


torch.Size([128, 13, 101]) torch.Size([128])


In [ ]:
# Varify Tensor's shape
# Example audio sample
audio = np.random.randn(16000).astype(np.float32)  # Simulate 1-second audio at 16kHz

# Compute MFCC features
mfcc_features = mfcc(y=audio, sr=16000, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, fmin=0, fmax=8000)

print(f'MFCC shape: {mfcc_features.shape}')  # Expected: (13, num_frames)

In [ ]:
import torch.nn as nn
import torch.optim as optim
from mamba_ssm import Mamba
from tqdm import tqdm

In [ ]:
# Define model
class KeywordSpottingModel(nn.Module):
    def __init__(self, input_dim, d_model, d_state, d_conv, expand):
        super(KeywordSpottingModel, self).__init__()
        self.proj = nn.Linear(input_dim, d_model)  # Initial projection layer
        self.mamba = Mamba(
            d_model=d_model,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        )
        self.fc = nn.Linear(d_model, len(label_names))  # Output layer


    def forward(self, x):
        x = x.permute(0, 2, 1)  # Reshape to [batch_size, num_frames, num_mfcc]
        x = self.proj(x)  # Project input to d_model dimension
        x = x.permute(0, 2, 1)  # Transpose to [batch_size, d_model, num_frames] for Mamba
        x = self.mamba(x)
        x = x.mean(dim=2)  # Global average pooling over the time dimension
        x = self.fc(x)
        return x

# Compute model size

In [ ]:
# Compute model size
import numpy as np

def get_flops_einsum(input_shapes, equation):
    np_arrs = [np.zeros(s) for s in input_shapes]
    optim = np.einsum_path(equation, *np_arrs, optimize="optimal")[1]
    for line in optim.split("\n"):
        if "optimized flop" in line.lower():
            flop = float(np.floor(float(line.split(":")[-1]) / 2))
            return flop

def flops_selective_scan_ref(B=1, L=256, D=768, N=16, with_D=True, with_Z=False, with_Group=True, with_complex=False):
    flops = 0
    flops += get_flops_einsum([[B, D, L], [D, N]], "bdl,dn->bdln")
    if with_Group:
        flops += get_flops_einsum([[B, D, L], [B, N, L], [B, D, L]], "bdl,bnl,bdl->bdln")
    else:
        flops += get_flops_einsum([[B, D, L], [B, D, N, L], [B, D, L]], "bdl,bdnl,bdl->bdln")

    in_for_flops = B * D * N
    if with_Group:
        in_for_flops += get_flops_einsum([[B, D, N], [B, D, N]], "bdn,bdn->bd")
    else:
        in_for_flops += get_flops_einsum([[B, D, N], [B, N]], "bdn,bn->bd")
    flops += L * in_for_flops

    if with_D:
        flops += B * D * L
    if with_Z:
        flops += B * D * L

    return flops


In [ ]:
def calculate_SSM_flops(model, x, y):
    B, D, L = x[0].shape
    N = model.d_state
    flops = flops_selective_scan_ref(B=B, L=L, D=D, N=N, with_D=True, with_Z=False, with_Group=True, with_complex=False)
    model.total_ops += torch.DoubleTensor([flops])


In [30]:
# Initialize model, loss function, and optimizer
input_dim = 13  # Number of MFCC features
d_model = 101  # Number of frames
d_state = 16
d_conv = 4
expand = 2

model = KeywordSpottingModel(input_dim=input_dim, d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand).to("cuda")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
import thop
# Register custom operation
macs, params, ret_layer_info = thop.profile(model, inputs=(torch.randn(128, 13, 101).to("cuda"),)
,custom_ops={Mamba: calculate_SSM_flops},report_missing=True, ret_layer_info=True)
print()
print(f"MACs: {macs} Which are {macs/1e9} Giga-MACs, Params: {params}")


[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.activation.SiLU'>. Treat it as zero Macs and zero Params.
[INFO] Customize rule calculate_SSM_flops() <class 'mamba_ssm.modules.mamba_simple.Mamba'>.
[WARN] Cannot find rule for <class '__main__.KeywordSpottingModel'>. Treat it as zero Macs and zero Params.

MACs: 91502114.0 Which are 0.091502114 Giga-MACs, Params: 77374.0


In [33]:
print(ret_layer_info)

{'proj': (16974464.0, 1414.0, {}), 'mamba': (74424226.0, 75144.0, {}), 'fc': (103424.0, 816.0, {})}


In [37]:
# Training loop
num_epochs = 100

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for audio, labels in tqdm(train_loader, total=len(train_loader)):
        audio, labels = audio.to("cuda"), labels.to("cuda")

        # Forward pass
        outputs = model(audio)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Training Accuracy: {train_accuracy}%')

    train_accuracies.append(train_accuracy)
    train_losses.append(running_loss/len(train_loader))

    # Validation
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():
        for audio, labels in val_loader:
            audio, labels = audio.to("cuda"), labels.to("cuda")
            outputs = model(audio)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f'Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {val_accuracy}%')

    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss/len(val_loader))


100%|███████████████████████████████████████████| 50/50 [00:14<00:00,  3.44it/s]


Epoch 1/100, Training Loss: 1.9532436728477478, Training Accuracy: 25.640625%
Validation Loss: 1.9110994614087617, Validation Accuracy: 26.0%


100%|███████████████████████████████████████████| 50/50 [00:14<00:00,  3.48it/s]


Epoch 2/100, Training Loss: 1.8992481327056885, Training Accuracy: 28.6875%
Validation Loss: 1.8720366404606745, Validation Accuracy: 29.1875%


100%|███████████████████████████████████████████| 50/50 [00:14<00:00,  3.49it/s]


...

Epoch 65/100, Training Loss: 0.5161435723304748, Training Accuracy: 82.125%
Validation Loss: 0.6230013851936047, Validation Accuracy: 78.4375%


100%|███████████████████████████████████████████| 50/50 [00:09<00:00,  5.38it/s]


Epoch 66/100, Training Loss: 0.5060908252000809, Training Accuracy: 82.90625%
Validation Loss: 0.6083644582675054, Validation Accuracy: 78.9375%


 24%|██████████▎                                | 12/50 [00:02<00:07,  5.30it/s]


KeyboardInterrupt: 

# Training loop

In [ ]:
# Training loop
num_epochs = 100

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for audio, labels in tqdm(train_loader, total=len(train_loader)):
        audio, labels = audio.to("cuda"), labels.to("cuda")

        # Forward pass
        outputs = model(audio)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Training Accuracy: {train_accuracy}%')

    train_accuracies.append(train_accuracy)
    train_losses.append(running_loss/len(train_loader))

    # Validation
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():
        for audio, labels in val_loader:
            audio, labels = audio.to("cuda"), labels.to("cuda")
            outputs = model(audio)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f'Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {val_accuracy}%')

    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss/len(val_loader))


In [ ]:
# Function for plotting learning curves
from matplotlib import pyplot as plt

def plot_learning_curves(train_accuracies, val_accuracies, train_losses, val_losses):
  epochs = range(1,len(train_accuracies)+1)

  plt.plot(epochs, train_accuracies, 'r', label='Training accuracy')
  plt.plot(epochs, val_accuracies, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend()
  yticks = np.arange(0, 101, 5)
  plt.yticks(yticks)
  plt.grid(True)
  plt.show()
  plt.clf()

  plt.plot(epochs, train_losses, 'r', label='Training Loss')
  plt.plot(epochs, val_losses, 'b', label='Validation Loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.grid(True)
  plt.show()

In [ ]:
# Plot the learning curves
plot_learning_curves(train_accuracies, val_accuracies, train_losses, val_losses)

# With L2 regulariztion :

In [ ]:
# Initialize model, loss function, and optimizer
input_dim = 13  # Number of MFCC features
d_model = 101  # Number of frames
d_state = 16
d_conv = 4
expand = 2

model = KeywordSpottingModel(input_dim=input_dim, d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand).to("cuda")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) # weight_decay for L2 regulariztopn

In [ ]:
# Training loop
num_epochs = 100

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for audio, labels in tqdm(train_loader, total=len(train_loader)):
        audio, labels = audio.to("cuda"), labels.to("cuda")

        # Forward pass
        outputs = model(audio)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Training Accuracy: {train_accuracy}%')

    train_accuracies.append(train_accuracy)
    train_losses.append(running_loss/len(train_loader))

    # Validation
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():
        for audio, labels in val_loader:
            audio, labels = audio.to("cuda"), labels.to("cuda")
            outputs = model(audio)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f'Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {val_accuracy}%')

    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss/len(val_loader))

In [ ]:
# Plot the learning curves
plot_learning_curves(train_accuracies, val_accuracies, train_losses, val_losses)

# Adding Droput layer

In [ ]:
# Define model
class KeywordSpottingModel(nn.Module):
    def __init__(self, input_dim, d_model, d_state, d_conv, expand):
        super(KeywordSpottingModel, self).__init__()
        self.proj = nn.Linear(input_dim, d_model)  # Initial projection layer
        self.mamba = Mamba(
            d_model=d_model,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        )
        self.fc = nn.Linear(d_model, len(label_names))  # Output layer

        self.dropout = nn.Dropout(0.5)  # Dropout layer with a dropout rate of 0.5


    def forward(self, x):
        x = x.permute(0, 2, 1)  # Reshape to [batch_size, num_frames, num_mfcc]
        x = self.proj(x)  # Project input to d_model dimension
        x = x.permute(0, 2, 1)  # Transpose to [batch_size, d_model, num_frames] for Mamba
        x = self.mamba(x)
        x = self.dropout(x)  # Apply dropout after Mamba
        x = x.mean(dim=2)  # Global average pooling over the time dimension
        x = self.fc(x)
        return x

In [ ]:
# Initialize model, loss function, and optimizer
input_dim = 13  # Number of MFCC features
d_model = 101  # Number of frames
d_state = 16
d_conv = 4
expand = 2

model = KeywordSpottingModel(input_dim=input_dim, d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand).to("cuda")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) # weight_decay for L2 regulariztopn

In [ ]:
# Training loop
num_epochs = 100

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for audio, labels in tqdm(train_loader, total=len(train_loader)):
        audio, labels = audio.to("cuda"), labels.to("cuda")

        # Forward pass
        outputs = model(audio)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Training Accuracy: {train_accuracy}%')

    train_accuracies.append(train_accuracy)
    train_losses.append(running_loss/len(train_loader))

    # Validation
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():
        for audio, labels in val_loader:
            audio, labels = audio.to("cuda"), labels.to("cuda")
            outputs = model(audio)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    print(f'Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {val_accuracy}%')

    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss/len(val_loader))

In [ ]:
# Plot the learning curves
plot_learning_curves(train_accuracies, val_accuracies, train_losses, val_losses)